In [27]:
import pandas as pd
import finpy_tse as fpy
import psycopg
from psycopg import Cursor
from sqlalchemy import create_engine
import jdatetime as jdt
import numpy as np
import re
from .CONFIG import DB_PASS as PASS

In [44]:
# Connect to an existing database
with psycopg.connect(f"dbname=metafid user=postgres password={PASS}") as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        # Execute a command: this creates a new table
        cur.execute("""
        CREATE TABLE "sigma" (
            "ua" varchar PRIMARY KEY,
            "sigma" decimal NOT NULL
            );
        ALTER TABLE "sigma" ADD FOREIGN KEY ("ua") REFERENCES "ua" ("ua");
        """)

        # Make the changes to the database persistent
        conn.commit()


In [ ]:

with psycopg.connect("dbname=metafid user=postgres password=*****") as conn:
    # Open a cursor to perform database operations
    with conn.cursor() as cur:
        with cur.copy("COPY ua (ua) FROM STDIN") as copy:
            for record in ua_tuple:
                copy.write_row(record)
        # Make the changes to the database persistent
        conn.commit()

In [48]:
def insert_data(table,cols, records):
    with psycopg.connect(f"dbname=metafid user=postgres password={PASS}") as conn:
    # Open a cursor to perform database operations
        with conn.cursor() as cur:
            with cur.copy(f"COPY {table} {cols} FROM STDIN") as copy:
                for record in records:
                    copy.write_row(record)
            # Make the changes to the database persistent
            conn.commit()

In [47]:
def query(cols, table):
    with psycopg.connect(f"dbname=metafid user=postgres password={PASS}") as conn:
    # Open a cursor to perform database operations
        with conn.cursor() as cur:
            cur.execute(f"SELECT {cols} FROM {table}")
            cur.fetchone()
            conn.commit()
            return [i[0] for i in cur]

In [24]:
def query_df(cols, table):
    with psycopg.connect(f"dbname=metafid user=postgres password={PASS}") as conn:
    # Open a cursor to perform database operations
        with conn.cursor() as cur:
            cur.execute(f"SELECT {cols} FROM {table}")
            cur.fetchone()
            conn.commit()
            return pd.DataFrame([i[0] for i in cur], columns=re.findall("[a-z A-Z]+", cols))

In [10]:
def drop(table, condition):
    with psycopg.connect(f"dbname=metafid user=postgres password={PASS}") as conn:
    # Open a cursor to perform database operations
        with conn.cursor() as cur:
            cur.execute(f"DELETE FROM {table} WHERE {condition}")
            conn.commit()

In [11]:
drop("ua", "ua='ت ثاميد'")

In [12]:
ua = query("ua", "ua")

In [ ]:
today = str(jdt.date.today())
ago_700 = str(jdt.date.today() - jdt.timedelta(days=700))
ua_hist = pd.DataFrame()
for ticker in ua:
    print(ticker)
    df = (
        fpy.Get_Price_History(
            stock=ticker,
            start_date=ago_700,
            end_date=today,
            ignore_date=False,
            adjust_price=True,
            show_weekday=False,
            double_date=True,
        )
        .reset_index()
        .rename(
            columns=lambda x: str.lower(x.replace(" ", "_").replace("-", "_"))
        )
    )
    
    cols = ["date", "ticker"]+["adf"]
    ua_hist = pd.concat([ua_hist, df])

In [63]:
cols = ["date", "ticker"]+[col for col in ua_hist.columns if "adj" in col ]
cols_db = "(date, ua, open, high, low, close, final)"
ua_hist = ua_hist[cols]

In [61]:

ua_hist_list = list(ua_hist.itertuples(index=False, name=None))

In [67]:
insert_data(table="ua_hist", cols=cols_db, records=ua_hist_list)

In [53]:
query(cols=cols_db, table='ua_hist')

[]

In [26]:
df = query_df("(ua,final)", "ua_hist")

In [36]:
def sigma(price:pd.Series):
    # calc sigma
    pct_change = np.log(1 + price.pct_change())
    return pct_change.std()

In [45]:
groups = df.groupby(by="ua")
sigma_list = []
for name, group in groups:
    sigma_list.append((name,sigma(group.final[-100:].astype(int))))
    

In [49]:
insert_data("sigma", "(ua, sigma)", sigma_list)